__This code simulates a Crisis and Emergency Shelter System as a M/M/s/{K}+M QuEuE.__
- Warm up is 150 days and fills the shelter up to 90% (considering the arrival rate of 4.44)
- Number of servers (n) in a queue is the number of beds in a shelter 
- Service time in system is the length of stay of youth, exponentially distributed
- Youth profiles (demographic profiles) come from excel files
- If youth received bed, we assume they receive the service as well


In [1]:
#Libraries
import numpy as np
import pandas as pd
import random
from numpy import random
import openpyxl
import xlrd
import time

from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
#################################################
##  Warm up! Fill the shelter within 150 days  ##
#################################################

#Warm-up period fills the shelter up t0 90% capacity, so that we have a realistic starting point to our simulation

warmup_days =150
arrival_per_day = 4.44
warmup_youth_number= round(warmup_days*arrival_per_day)

df_warm = pd.read_excel(r'<your directory>\Priority_WarmUp.xlsx')  
df_warm = df_warm.iloc[:warmup_youth_number,1:]


In [34]:
#########################
##   Creating youths   ##
#########################

#Time frame
num_days = 360 # 360 is the simulation length after the warm-up
arrival_per_day = 4.44
total_arrival = round(arrival_per_day*num_days)

#Number of youths
Y= total_arrival

#Get data from excel sheets for youth profiles
xls = pd.ExcelFile(r'<your directory>\YouthProfiles_20Replications.xlsx')
sheet_names = xls.sheet_names

#Number of replications (in this case we included 20 replications)
reps = len(sheet_names)
print('There are ' +str(reps)+ ' replications in this simulation')

#Shelter beds (total number of crisis emergency beds required in this shelter to provide high service quality to RHY)
bed = 270

threshold_group6 = 25

# Set parameters for the model #

# Service time is :length of stay (exponential)
avg_service_time= 60
#servicetime = random.exponential(scale=avg_service_time, size=1)

# Patience time (exponential)
avg_patience = 2
#patiencetime = random.exponential(scale=avg_service_time, size=1)

#Results will have the average wait time, abandonment proportions and so on of replications
results = {}


There are 20 replications in this simulation


In [35]:
tic = time.perf_counter()
for h in range(0,reps):
    
    df_youth = pd.read_excel(r'<your directory>\YouthProfiles_20Replications.xlsx' , sheet_name = sheet_names[h])
    df_youth = df_youth.iloc[warmup_youth_number:warmup_youth_number+Y,1:]
    
    ###  Arrival Process ###

    #Poisson, exponential inter-arrival times (the plus 45 adds the warm-up time arrivals to the picture)
    arrival_time = np.cumsum(random.exponential(scale=1/total_arrival ,size=(total_arrival, 1)))*num_days+warmup_days    

    #Combine, Vulnerability Group of youth and arrival time
    arriving_profiles = {}
    for y in range(Y):
                                 #           no            HT Victim              substance          gender               juvie               minority        vulnerability group    arrival time                             
        arriving_profiles[y] = [y+warmup_youth_number, df_youth.iloc[y,0], df_youth.iloc[y,1], df_youth.iloc[y,2] , df_youth.iloc[y,3], df_youth.iloc[y,4], df_youth.iloc[y,5], arrival_time[y]]

    
    #Define 2 lists that are going to help us keep track of the serviced youth, and unserviced youth
    serviced_list =[]
    non_serviced_list =[]

    # Initialize the code by turning df_warm into df_serviced
    serviced_list= df_warm.values.tolist()

    #for i in range(warmup_youth_number, warmup_youth_number+Y):
    i= warmup_youth_number # to keep track of serviced list 
    j= 0                   # to keep track of arriving youth

    while len(arriving_profiles)>0:  
        #if number of beds open in the system is larger than 25, all youth can enter the system with no prioritizing
        if serviced_list[-1][16]> threshold_group6:

            arrivals_no_priority = []
            for k in arriving_profiles.keys():
                arrivals_no_priority.append(arriving_profiles[k]) 

            #characteristics of arriving youth
            number = arrivals_no_priority[j][0]
            HT= arrivals_no_priority[j][1]
            substance = arrivals_no_priority[j][2]
            gender = arrivals_no_priority[j][3]
            juvie = arrivals_no_priority[j][4]
            minority = arrivals_no_priority[j][5]
            group =  arrivals_no_priority[j][6]       

            #times of arriving youth
            arrival_time = arrivals_no_priority[j][7]
            service_time = min(random.exponential(scale=avg_service_time, size=1)[0], 90)          #length of stay of youth
            patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            

            #find the number of youth in the system
            num_in_system_arrivals = []
            for k in range(0,i):
                num_in_system_arrivals.append(serviced_list[k][7])

            num_in_system_departures = []
            for k in range(0,i):
                num_in_system_departures.append(serviced_list[k][11])

            #number in system is equal to number of people who arrived till then minus the number of people who left the system
            #note that i include the youth arriving to the system as well (+1)
            number_in_system = sum(num_in_system_arrivals[0:i]<arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival

            tmp = sorted(num_in_system_departures)
            #if arrival is later than the first possible departure, service should start at the arrival time
            if arrival_time > tmp[len(serviced_list)- bed]:
                start_service_time = arrival_time                                  #time youth starts service

            #if arrival is before the departure, service should start at the departure time
            else:
                start_service_time = tmp[len(serviced_list)- bed]              #time youth starts service

            departure_time = start_service_time + service_time                #time youth leaves system
            wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
            num_open_beds = max(bed - number_in_system,0)                     #number of open beds
            utilization = (1-num_open_beds/bed)                               #utilization during the time youth is staying

            if wait_time >= patience_time:
                #create a list for the un-served youth
                actual_wait_time= patience_time                                #updated wait time based on abandonment
                received_service = 0                                           #binary variable that shows whether youth received service or not

                non_serviced_stacked = [number, HT, substance, gender, juvie, minority, group, arrival_time, service_time, 
                                patience_time, start_service_time, departure_time, wait_time, number_in_system , 
                                actual_wait_time, received_service, num_open_beds, utilization]
                non_serviced_list.append(non_serviced_stacked)

                #now we have to delete the youth that has been appended to non-serviced list from the arriving list
                for k in arriving_profiles.keys():
                    if arriving_profiles[k][0] == number:
                        del arriving_profiles[k]
                        break


            else:
                actual_wait_time= wait_time                                     #updated wait time based on abandonment
                received_service = 1                                            #binary variable that shows whether youth received service or not
                #create a list for the serviced youths
                serviced_stacked = [number, HT, substance, gender, juvie, minority, group, arrival_time, service_time, 
                                    patience_time, start_service_time, departure_time, wait_time, number_in_system , 
                                    actual_wait_time, received_service, num_open_beds, utilization]
                serviced_list.append(serviced_stacked)

                #now we have to delete the youth that has been appended to serviced list from the arriving list
                for k in arriving_profiles.keys():
                    if arriving_profiles[k][0] == number:
                        del arriving_profiles[k]
                        break

                #if a youth received service, increase i by 1
                i= i+1


        ########################################

        #if number of beds open in the system is less than 25, youth from group 6 cannot enter the system
        if serviced_list[-1][16]<= threshold_group6:
            group6_arrivals =[]
            arrivals_without_group6 = []
            arrivals_with_group6_behind = []

            for k in arriving_profiles.keys():
                if arriving_profiles[k][6]==6:
                    group6_arrivals.append(arriving_profiles[k])
                else:
                    arrivals_without_group6.append(arriving_profiles[k])

            #append group 6 arrivals to the back
            arrivals_with_group6_behind = arrivals_without_group6 + group6_arrivals

            if len(arrivals_with_group6_behind) == 0:
                break

            else:

                #characteristics of arriving youth
                number = arrivals_with_group6_behind[j][0]
                HT= arrivals_with_group6_behind[j][1]
                substance = arrivals_with_group6_behind[j][2]
                gender =arrivals_with_group6_behind[j][3]
                juvie = arrivals_with_group6_behind[j][4]
                minority = arrivals_with_group6_behind[j][5]
                group =  arrivals_with_group6_behind[j][6]       

                #times of arriving youth
                arrival_time = arrivals_with_group6_behind[j][7]
                service_time = min(random.exponential(scale=avg_service_time, size=1)[0], 90)  #length of stay of youth
                patience_time = random.exponential(scale=avg_patience, size=1)[0]        #patience time            

                #find the number of youth in the system
                num_in_system_arrivals = []
                for k in range(0,i):
                    num_in_system_arrivals.append(serviced_list[k][7])

                num_in_system_departures = []
                for k in range(0,i):
                    num_in_system_departures.append(serviced_list[k][11])

                #number in system is equal to number of people who arrived till then minus the number of people who left the system
                number_in_system = sum(num_in_system_arrivals[0:i]<arrival_time)- sum(num_in_system_departures[0:i]<=arrival_time)+1              #number of youths in system at the arrival

                tmp = sorted(num_in_system_departures)
                #if arrival is later than the first possible departure, service should start at the arrival time
                if arrival_time > tmp[len(serviced_list)- bed]:
                    start_service_time = arrival_time                                  #time youth starts service

                #if arrival is before the departure, service should start at the departure time
                else:
                    start_service_time = tmp[len(serviced_list)- bed]              #time youth starts service

                departure_time = start_service_time + service_time                #time youth leaves system
                wait_time = start_service_time -arrival_time                      #amount of time youth waits                                                         
                num_open_beds = max(bed - number_in_system,0)                     #number of open beds
                utilization = (1-num_open_beds/bed)                               #utilization during the time youth is staying

                if wait_time >= patience_time:
                    #create a list for the un-served youth
                    actual_wait_time= patience_time                                #updated wait time based on abandonment
                    received_service = 0                                           #binary variable that shows whether youth received service or not

                    non_serviced_stacked = [number, HT, substance, gender, juvie, minority, group, arrival_time, service_time, 
                                    patience_time, start_service_time, departure_time, wait_time, number_in_system , 
                                    actual_wait_time, received_service, num_open_beds, utilization]
                    non_serviced_list.append(non_serviced_stacked)


                    #now we have to delete the youth that has been appended to non-serviced list from the arriving list
                    for k in arriving_profiles.keys():
                        if arriving_profiles[k][0] == number:
                            del arriving_profiles[k]
                            break


                else:
                    #create a list for the serviced youths
                    actual_wait_time= wait_time                                     #updated wait time based on abandonment
                    received_service = 1                                            #binary variable that shows whether youth received service or not

                    serviced_stacked = [number, HT, substance, gender, juvie, minority, group, arrival_time, service_time, 
                                        patience_time, start_service_time, departure_time, wait_time, number_in_system , 
                                        actual_wait_time, received_service, num_open_beds, utilization]
                    serviced_list.append(serviced_stacked)

                    #now we have to delete the youth that has been appended to serviced list from the arriving list
                    for k in arriving_profiles.keys():
                        if arriving_profiles[k][0] == number:
                            del arriving_profiles[k]
                            break

                    #if a youth received service, increase i by 1
                    i= i+1



    #Turn serviced and non-serviced lists into pandas data frames
    serviced_df = pd.DataFrame(serviced_list, columns =['No', 'HT Victim', 'substance', 'gender ', 'juvie', 'minority', 
                                                            'vulnerability group', 'arrival time', 'service time', 'patience time', 
                            'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num open beds','utilization'])

    non_serviced_df = pd.DataFrame(non_serviced_list, columns =['No', 'HT Victim', 'substance', 'gender ', 'juvie', 'minority', 
                                                            'vulnerability group', 'arrival time', 'service time', 'patience time', 
                            'start service time', 'departure time', 'wait time', 'number in system' , 'actual wait time', 'received service', 'num open beds','utilization']) 

    
    #remove warm-up
    serviced_without_warmup = serviced_df.iloc[warmup_youth_number:]

    #System Abandonment and Average Wait
    prop_abandoned = len(non_serviced_df)/(len(non_serviced_df)+len(serviced_without_warmup))
    avg_wait_serviced = serviced_without_warmup['actual wait time'].mean(axis=0)
    avg_wait_nonserviced = non_serviced_df['actual wait time'].mean(axis=0)
    df = serviced_without_warmup['actual wait time'].append(non_serviced_df['actual wait time'])
    avg_wait_system = df.mean(axis=0)

    #System Utilization
    utilization_overall = (serviced_without_warmup.iloc[1]['start service time'] - serviced_df.iloc[warmup_youth_number]['start service time'])* serviced_without_warmup.iloc[1]['utilization'] 

    for i in range(2, len(serviced_without_warmup)):
        tmp = (serviced_without_warmup.iloc[i]['start service time'] - serviced_without_warmup.iloc[i-1]['start service time'])
        util_timeframe = tmp*serviced_without_warmup.iloc[i]['utilization'] 
        utilization_overall = utilization_overall + util_timeframe 

    utilization_system = utilization_overall/(serviced_without_warmup.iloc[-1]['start service time'] - serviced_without_warmup.iloc[1]['start service time']) 

    ##############

    #Abandonment Rate based on the vulnerability groups
    prop_abandoned_A = len(non_serviced_df[non_serviced_df['vulnerability group'] == 1])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 1])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 1]))
    prop_abandoned_B = len(non_serviced_df[non_serviced_df['vulnerability group'] == 2])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 2])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 2]))
    prop_abandoned_C = len(non_serviced_df[non_serviced_df['vulnerability group'] == 3])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 3])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 3]))
    prop_abandoned_D = len(non_serviced_df[non_serviced_df['vulnerability group'] == 4])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 4])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 4]))
    prop_abandoned_E = len(non_serviced_df[non_serviced_df['vulnerability group'] == 5])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 5])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 5]))
    prop_abandoned_F = len(non_serviced_df[non_serviced_df['vulnerability group'] == 6])/ (len(non_serviced_df[non_serviced_df['vulnerability group'] == 6])+ len(serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 6]))

    #Wait Times based on the vulnerability groups
    avg_wait_A = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 1]['actual wait time'].mean(axis=0)
    avg_wait_B = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 2]['actual wait time'].mean(axis=0)
    avg_wait_C = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 3]['actual wait time'].mean(axis=0)
    avg_wait_D = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 4]['actual wait time'].mean(axis=0)
    avg_wait_E = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 5]['actual wait time'].mean(axis=0)
    avg_wait_F = serviced_without_warmup[serviced_without_warmup['vulnerability group'] == 6]['actual wait time'].mean(axis=0)
    
    results[h] = [prop_abandoned,avg_wait_serviced, avg_wait_nonserviced, avg_wait_system, utilization_system, 
                 prop_abandoned_A, prop_abandoned_B, prop_abandoned_C, prop_abandoned_D, prop_abandoned_E, prop_abandoned_F, 
                 avg_wait_A, avg_wait_B, avg_wait_C, avg_wait_D, avg_wait_E, avg_wait_F]
    
toc =time.perf_counter()


print(f"\n-----------------------------------------------------")
print(f'Running the simulation with '+ str(reps)+ ' replications takes '+str(toc - tic)+' seconds')
print(f"\n-----------------------------------------------------")


-----------------------------------------------------
Running the simulation with 20 replications takes 228.3694730000002 seconds

-----------------------------------------------------


__Each value in serviced list respresents something about youth, and they are given below:__
- serviced_list[i][0] is: youth number
- serviced_list[i][1] is: HT Victim	status
- serviced_list[i][2] is: substance abuse status
- serviced_list[i][3] is: gender	
- serviced_list[i][4] is: juvie	
- serviced_list[i][5] is: minority status
- serviced_list[i][6] is: vulnerability group based on the status above	
- serviced_list[i][7] is: arrival time of youth
- serviced_list[i][8] is: service time of youth
- serviced_list[i][9] is: patience time	of youth
- serviced_list[i][10] is: start service time of youth
- serviced_list[i][11] is: departure time of youth
- serviced_list[i][12] is: wait time of youth	
- serviced_list[i][13] is: number in system	
- serviced_list[i][14] is: actual wait time	(when patience come into play, youth actually only wait till their patience ends)
- serviced_list[i][15] is: whether youth received service or not (binary)	
- serviced_list[i][16] is: num open beds in the system

In [36]:
### Summary of the Simulation ###
#Queueing performance metrics are given for the whole system and per youth group #

sum_system_abandonment = 0
sum_avg_wait_serviced = 0
sum_avg_wait_nonserviced =0
sum_avg_wait_system = 0
sum_utilization_system =0
sum_prop_abandoned_A =0
sum_prop_abandoned_B =0
sum_prop_abandoned_C =0
sum_prop_abandoned_D =0
sum_prop_abandoned_E = 0
sum_prop_abandoned_F =0
sum_avg_wait_A =0
sum_avg_wait_B =0
sum_avg_wait_C =0
sum_avg_wait_D =0
sum_avg_wait_E =0
sum_avg_wait_F =0

for h in range(0,10):
    sum_system_abandonment = sum_system_abandonment + results[h][0]
    sum_avg_wait_serviced = sum_avg_wait_serviced + results[h][1]
    sum_avg_wait_nonserviced = sum_avg_wait_nonserviced + results[h][2]
    sum_avg_wait_system = sum_avg_wait_system + results[h][3]
    sum_utilization_system = sum_utilization_system + results[h][4]
    sum_prop_abandoned_A = sum_prop_abandoned_A + results[h][5]
    sum_prop_abandoned_B = sum_prop_abandoned_B + results[h][6]
    sum_prop_abandoned_C = sum_prop_abandoned_C + results[h][7]
    sum_prop_abandoned_D = sum_prop_abandoned_D + results[h][8]
    sum_prop_abandoned_E = sum_prop_abandoned_E + results[h][9]
    sum_prop_abandoned_F = sum_prop_abandoned_F + results[h][10]
    sum_avg_wait_A = sum_avg_wait_A + results[h][11]
    sum_avg_wait_B = sum_avg_wait_B + results[h][12]
    sum_avg_wait_C = sum_avg_wait_C + results[h][13]
    sum_avg_wait_D = sum_avg_wait_D + results[h][14]
    sum_avg_wait_E = sum_avg_wait_E + results[h][15]
    sum_avg_wait_F = sum_avg_wait_F + results[h][16]
    
avg_system_abandonment = sum_system_abandonment/reps
avg_avg_wait_serviced = sum_avg_wait_serviced/reps
avg_avg_wait_nonserviced = sum_avg_wait_nonserviced/reps
avg_avg_wait_system = sum_avg_wait_system /reps
avg_utilization_system = sum_utilization_system/reps
avg_prop_abandoned_A = sum_prop_abandoned_A/reps
avg_prop_abandoned_B = sum_prop_abandoned_B/reps
avg_prop_abandoned_C = sum_prop_abandoned_C/reps
avg_prop_abandoned_D = sum_prop_abandoned_D/reps
avg_prop_abandoned_E = sum_prop_abandoned_E/reps
avg_prop_abandoned_F = sum_prop_abandoned_F/reps
avg_avg_wait_A = sum_avg_wait_A/reps
avg_avg_wait_B = sum_avg_wait_B/reps
avg_avg_wait_C = sum_avg_wait_C/reps 
avg_avg_wait_D = sum_avg_wait_D/reps
avg_avg_wait_E = sum_avg_wait_E/reps
avg_avg_wait_F = sum_avg_wait_F/reps


print('-------------------')
print('__Overall Outputs__')
print('-------------------')
print('-Abandonment')
print('Overall {:.4}'.format(avg_system_abandonment*100)+'% of youth has abandoned the system ')
print('')
print('-Utilization')
print('System utilization is {:.4}'.format(avg_utilization_system*100)+'%')
print('')
print('-Average Wait')
print('On average youth wait {:.2}'.format(avg_avg_wait_system)+' days.')
print('Youth who have received services on average wait {:.2}'.format(avg_avg_wait_serviced)+' days.')
print('Youth who have not received services on average wait {:.2}'.format(avg_avg_wait_nonserviced)+' days.')


##############

#Abandonment Rate based on the vulnerability groups
print('---------------------')
print('__Per Group Outputs__')
print('---------------------')
print('-Abandonment')
print('From Group A: {:.4}'.format(avg_prop_abandoned_A*100)+'% of youth has abandoned the system ')
print('From Group B: {:.4}'.format(avg_prop_abandoned_B*100)+'% of youth has abandoned the system ')
print('From Group C: {:.4}'.format(avg_prop_abandoned_C*100)+'% of youth has abandoned the system ')
print('From Group D: {:.4}'.format(avg_prop_abandoned_D*100)+'% of youth has abandoned the system ')
print('From Group E: {:.4}'.format(avg_prop_abandoned_E*100)+'% of youth has abandoned the system ')
print('From Group F: {:.4}'.format(avg_prop_abandoned_F*100)+'% of youth has abandoned the system ')
print('')
print('-Average Wait')
print('On average a serviced youth from Group A waits {:.4}'.format(avg_avg_wait_A)+' days')
print('On average a serviced youth from Group B waits {:.4}'.format(avg_avg_wait_B)+' days')
print('On average a serviced youth from Group C waits {:.4}'.format(avg_avg_wait_C)+' days')
print('On average a serviced youth from Group D waits {:.4}'.format(avg_avg_wait_D)+' days')
print('On average a serviced youth from Group E waits {:.4}'.format(avg_avg_wait_E)+' days')
print('On average a serviced youth from Group F waits {:.4}'.format(avg_avg_wait_F)+' days')

print('---------------------')


-------------------
__Overall Outputs__
-------------------
-Abandonment
Overall 0.0% of youth has abandoned the system 

-Utilization
System utilization is 77.4%

-Average Wait
On average youth wait 2e-05 days.
Youth who have received services on average wait 2e-05 days.
Youth who have not received services on average wait nan days.
---------------------
__Per Group Outputs__
---------------------
-Abandonment
From Group A: 0.0% of youth has abandoned the system 
From Group B: 0.0% of youth has abandoned the system 
From Group C: 0.0% of youth has abandoned the system 
From Group D: 0.0% of youth has abandoned the system 
From Group E: 0.0% of youth has abandoned the system 
From Group F: 0.0% of youth has abandoned the system 

-Average Wait
On average a serviced youth from Group A waits 0.0 days
On average a serviced youth from Group B waits 4.878e-05 days
On average a serviced youth from Group C waits 4.83e-05 days
On average a serviced youth from Group D waits 0.0 days
On average 